In [2]:
# cifar10.ipynb
from src.simulate import simulate
from src.helper import display_train_stats, ExperimentLogger, plot_accuracy
from src.config import *

dataset = 'CIFAR10'
img_dir = f'{img_dir_root}/{dataset}'
c, b, e = 10, 50, 5
t_acc = target_accuracy_set[dataset]
label_parameters = f"C={c/100: .2f}, B={b},  E={e}, ACC={t_acc}"
stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=1, learning_rate=0.03, target_accuracy=t_acc, iid=True, mlp=False)
name = f"{dataset}_{kw}_{c}_{b}_{e}"
fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=t_acc)
fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')


Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10 Loaded!
Round 1 | ids: [4, 19, 24, 27, 49, 50, 56, 69, 78, 95] | average accuracy: 0.1417 | train loss: 1.7485 | test loss: 2.2380


NameError: name 'stats_dir' is not defined